In [1]:
from pathlib import Path
import urllib
import timm
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import torch

/Users/aaronestes/anaconda3/envs/hello_world_poetry/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_folder = 'test_images'
cwd = Path.cwd()
f = 'alex_dog_german_shepherd.jpg'
filename = Path.home().joinpath(cwd, base_folder, f)
print(filename)

## TODO: the image should be from the camera
# filename = str(input())

/Users/aaronestes/projects/Hello_World/Poetry/test_images/alex_dog_german_shepherd.jpg


In [3]:
model = timm.create_model('inception_v3', pretrained=True)

In [4]:
model.eval()

InceptionV3(
  (Conv2d_1a_3x3): ConvNormAct(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (Conv2d_2a_3x3): ConvNormAct(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (Conv2d_2b_3x3): ConvNormAct(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
  )
  (Pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): ConvNormAct(
    (conv): Conv2d(64, 80, kernel_size

In [5]:
config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [6]:
img = Image.open(filename).convert('RGB')

In [7]:
tensor = transform(img).unsqueeze(0)

In [8]:
with torch.no_grad():
    out = model(tensor)

probabilities = torch.nn.functional.softmax(out[0], dim=0)
print(probabilities.shape)

torch.Size([1000])


In [9]:
imagenet_classes_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_classes_filename = "imagenet_classes.txt"

url, filename = (imagenet_classes_url, imagenet_classes_filename)

urllib.request.urlretrieve(url, filename)

with open(imagenet_classes_filename, "r") as f:
    categories = [s.strip() for s in f.readlines()]

top5_prob, top5_catid = torch.topk(probabilities, 5)

for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], str(round(float(top5_prob[i].item()*100), 1)) + "%")

German shepherd 71.0%
kelpie 8.0%
malinois 6.8%
bulletproof vest 2.3%
Norwegian elkhound 1.5%
